In [185]:
import numpy as np
import matplotlib.pyplot as mplt
from mathphys.functions import save, load
import os
from apsuite.optics_analysis.tune_correction import TuneCorr
from apsuite.orbcorr import OrbRespmat
from pymodels import si
import pyaccel as pa
from siriuspy.clientconfigdb import ConfigDBClient
from siriuspy.search import IDSearch
from siriuspy.devices import IVU, SOFB, IDFF
from siriuspy.search import IDSearch
import GPy as gpy

# Get orbit distortions

In [ ]:
# Get files
beamline = 'EMA'

files = os.listdir()
files = sorted(
    [file for file in files if 'orbit_variation' in file and 'new' not in file])
files

In [ ]:
devnameivu = IDSearch.conv_beamline_2_idname(beamline=beamline)
ivu = IVU(devnameivu)
idff = IDFF(ivu.devname)
corr_names = list()
corr_names.extend(idff.chnames)
corr_names.extend(idff.cvnames)

In [196]:
# Get orbit and gaps from files
orbsx = []
orbsy = []
for file in files:
    data = load(file)
    gaps = np.array(data['gap'])
    orbsx.append(data['orbx'])
    orbsy.append(data['orby'])

#  Average orbit over iteractions
orbx = np.array(orbsx).mean(axis=0)
orby = np.array(orbsy).mean(axis=0)

nr_pts = len(gaps)

# Get ref orb
ref_idx = gaps == 24
ref_orbx = orbx[ref_idx]
ref_orby = orby[ref_idx]

# Get orbit distortion for each gap
orbx_dist = orbx - ref_orbx
orby_dist = orby - ref_orby
orbd = np.hstack((orbx_dist, orby_dist))

# Correct distortions

In [ ]:
model = si.create_accelerator()
tunecorr = TuneCorr(model=model, acc='SI')
tunecorr.correct_parameters((49.16, 14.22))

### Calculate local corrs orbit response matrix

In [200]:
# Calculate local corrs orbit response matrix
orbmat = OrbRespmat(model=model, acc='SI', dim='6d', corr_system='SOFB')
allcorr_idcs = pa.lattice.find_indices(model, 'fam_name', 'IDC4')

if beamline == 'EMA':
    corr_idcs = allcorr_idcs[:2]
elif beamline == 'PAINEIRA':
    corr_idcs = allcorr_idcs[2:]

orbmat.ch_idx = corr_idcs
orbmat.cv_idx = corr_idcs
respmat = orbmat.get_respm()[:, :-1]


### Load local corrs orbit response matrix

In [ ]:
files = os.listdir()
files = sorted(
    [file for file in files if 'orbit_distortions_corrs' in file and 'new' not in file])
files

In [ ]:
data = load(files[0])
dcurr = data['delta_current']

respmat = np.zeros((320, 4))
for i, corr_name in enumerate(corr_names):
    dorbx = (data[corr_name]['pos']['dorbx'] - data[corr_name]['neg']['dorbx'])
    dorby = (data[corr_name]['pos']['dorby'] - data[corr_name]['neg']['dorby'])
    dorb = np.hstack((dorbx, dorby))
    respmat[:, i] = dorb/dcurr


### Perform correction

In [201]:
# load sofb respmat
sofb_respmat = np.array(ConfigDBClient(config_type='si_orbcorr_respm').get_config_value('ref_respmat'))

In [208]:
# Apply filter
# Correct orbit with SOFB
kick_s = np.linalg.pinv(sofb_respmat) @ orbd.T

# Keep only the kicks from the 4 SOFB corrs. closest to the ID
# Select corrector closest to the ID

nr_corrs = 4

fam_data = si.get_family_data(model)
if beamline == 'EMA':
    ivu_idc = fam_data['IVU18']['index'][0]
elif beamline == 'PAINEIRA':
    ivu_idc = fam_data['IVU18']['index'][1]
ivu_idc = int(np.array(ivu_idc).mean())

idcs_allch = fam_data['CH']['index']
idcs_allcv = fam_data['CV']['index']
idcs_allch = np.array(idcs_allch).reshape(len(idcs_allch))
idcs_allcv = np.array(idcs_allcv).reshape(len(idcs_allcv))

idcsclosest_ch = np.sort(np.argsort(np.abs(idcs_allch - ivu_idc), axis=0)[:nr_corrs])
idcsclosest_cv = np.sort(np.argsort(np.abs(idcs_allcv - ivu_idc), axis=0)[:nr_corrs])

idcs_corrsch_model = idcs_allch[idcsclosest_ch]
idcs_corrscv_model = idcs_allcv[idcsclosest_cv]

mask = np.zeros(len(kick_s))
mask[idcsclosest_ch] = 1
mask[len(idcs_allch)+idcsclosest_cv] = 1
mask = np.full((nr_pts, mask.shape[0]), mask)
mask = np.swapaxes(mask, 0, 1)

kick_s *= mask

# Calculate the Filtered Orbit Distortion (local, ID distortion)
orb_dist_f = (sofb_respmat @ kick_s).T

# Calculate the ID correctors kicks to correct the local distortions
corr_kicks = (np.linalg.pinv(respmat) @ orb_dist_f.T)

### IF necessary convert strength to current

In [ ]:
# If measured local corrs respmat was used
corr_currents = corr_kicks

In [225]:
from siriuspy.magnet.factory import NormalizerFactory

In [ ]:
corr_currents = np.zeros(corr_kicks.shape)
for i, corrname in enumerate(corr_names):
    n = NormalizerFactory.create(corrname.replace('PS', 'MA'))
    corr_currents[i, :] = n.conv_strength_2_current(corr_kicks[i, :], strengths_dipole=3)

# Gaussian process

In [ ]:
# Here 
ker = gpy.kern.RBF(input_dim=1)
ch1_kicks = corr_currents[0, :]
ch2_kicks = corr_currents[1, :]
cv1_kicks = corr_currents[2, :]
cv2_kicks = corr_currents[3, :]

gaps = np.linspace(4.2, 24, 200)
X = gaps.T
Y = np.vstack([ch1_kicks, ch2_kicks, cv1_kicks, cv2_kicks]).T
model = gpy.models.GPRegression(X, Y, ker)
model.optimize_restarts(num_restarts=10, num_processes=6, verbose=True)

y_pred, var = model.predict(X)

# Format Table

In [253]:
clt = ConfigDBClient(config_type='si_idff')
config = clt.get_config_value('ivu18_ref')
config

{'description': 'Config created with IDFFConfig.create_template_config',
 'polarizations': {'horizontal': {'ch1': [1, 0],
   'ch2': [2, 0],
   'cv1': [3, 0],
   'cv2': [4, 0],
   'kparameter': [4.2, 24],
   'lch': [5, 0],
   'pparameter': None}},
 'pvnames': {'ch1': 'SI-08SB:PS-CH-1:Current-SP',
  'ch2': 'SI-08SB:PS-CH-2:Current-SP',
  'cv1': 'SI-08SB:PS-CV-1:Current-SP',
  'cv2': 'SI-08SB:PS-CV-2:Current-SP',
  'kparameter': 'SI-08SB:ID-IVU18:Gap-Mon',
  'lch': 'SI-08SB:PS-LCH:Current-SP',
  'pparameter': None}}

In [258]:
config['description'] = 'Feedforward table. Measured on 2024/10/23. Only CHs and CVs'
config['polarizations']['horizontal']['ch1'] = -1*y_pred[0].tolist()
config['polarizations']['horizontal']['ch2'] = -1*y_pred[1].tolist()
config['polarizations']['horizontal']['cv1'] = -1*y_pred[2].tolist()
config['polarizations']['horizontal']['cv2'] = -1*y_pred[3].tolist()
config['polarizations']['horizontal']['cv2'] = np.zeros(len(gaps)).tolist()
config['polarizations']['horizontal']['kparameter'] = gaps.tolist()

{'ch1': [1, 0],
 'ch2': [2, 0],
 'cv1': [3, 0],
 'cv2': [4, 0],
 'kparameter': [4.2, 24],
 'lch': [5, 0],
 'pparameter': None}

In [ ]:
clt.check_valid_value(config)
clt.insert_config('ivu18_only_ch_cv', config)